## Linear Elasticity for Cantilever Beam

In [1]:
using Gridap, Gridap.CellData, SparseArrays, BenchmarkTools
using NearestNeighbors, Printf  # make sure this is at the top of your file
using Makie,GLMakie, LaTeXStrings, Makie.GeometryBasics

## 1. Problem
### Let's consider the 3D cantilever beam as follows:
<img src="imagines/3D_Cantilever.png" style="width:400px;height:250px"/>

The dimensions of beam are $L$ and $W$. 

This beam is fixed at $x=0$ and is applied a uniformly distributed load at $x=L$. The magnitude of the distributed load is $\mathbf{F} = [0, -1, 0]^T$. 

## Strong Form

**Equilibrium equation:**
$$
\nabla \cdot \boldsymbol{\sigma} + \mathbf{b} = \mathbf{0} \text{ in } \Omega,
$$

**Kinematics equation (strain–displacement relation):**
$$
\boldsymbol{\varepsilon} = \nabla_S \mathbf{u} = \frac{1}{2} \Big[ \nabla \mathbf{u} + (\nabla \mathbf{u})^T \Big].
$$

**Constitutive equation (stress–strain relation):**
$$
\boldsymbol{\sigma} = \mathbf{C} \boldsymbol{\varepsilon} 
\quad \text{or} \quad 
\boldsymbol{\sigma} = \lambda \, \text{tr}(\boldsymbol{\varepsilon})\, \mathbf{I} + 2\mu\, \boldsymbol{\varepsilon}.
$$

At the face with $x=L$ which is a boundary condition denoted by $\Gamma$, we have:

$$
\boldsymbol{\sigma} \cdot \mathbf{n} = \mathbf{F} = \begin{bmatrix}
0 & -1 & 0
\end{bmatrix}^T
 \text{ on } \Gamma,
$$

where $\boldsymbol{\sigma}$ is the stress tensor, $\mathbf{b}$ is the body force, $\mathbf{F}$ is the applied traction, and $\mathbf{n}$ is the outward unit normal.


## Derivation of the Weak Form from the Strong Form

### Step 1: Multiply by a Test Function
To derive the weak form, multiply the strong form by a test function $\mathbf{v} \in H^1(\Omega)$ (with $\mathbf{v} = \mathbf{0}$ on Dirichlet boundaries) and integrate over the domain $\Omega$:

$$
\int_{\Omega} (\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = -\int_{\Omega} \mathbf{b} \cdot \mathbf{v} \, d\Omega.
$$
In this example, we assume that $\mathbf{b} = \mathbf{0}$, the we have:

$$
\int_{\Omega} (\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = 0.
$$


### Step 2: Apply the Divergence Theorem
We have the equivalent form of $(\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} $ as follows:

$$
(\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} = \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) - \boldsymbol{\sigma} : \nabla \mathbf{v}.
$$
Then we have,
$$
\int_{\Omega} (\nabla_S^T \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = \int_{\Omega} \left[ \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) - \boldsymbol{\sigma} : \nabla \mathbf{v} \right] \, d\Omega.
$$

Note that $\{\cdot\}$ is the dot product and $\{:\}$ is the inner product.

Apply the divergence theorem to $ \int_{\Omega} \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) \, d\Omega$:

$$
\int_{\Omega} \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) \, d\Omega = \int_{\partial \Omega} (\boldsymbol{\sigma} \cdot \mathbf{v}) \cdot \mathbf{n} \, d\Gamma = \int_{\partial \Omega} (\boldsymbol{\sigma} \cdot \mathbf{n}) \cdot \mathbf{v} \, d\Gamma.
$$

Because, we set the boundary condition $\boldsymbol{\sigma} \cdot \mathbf{n} = \mathbf{F}$ on $\Gamma$, then we have:

$$
\int_{\partial \Omega} (\boldsymbol{\sigma} \cdot \mathbf{n}) \cdot \mathbf{v} \, d\Gamma = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma.
$$

Thus:

$$
\int_{\Omega} (\nabla_S^T \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma - \int_{\Omega} \boldsymbol{\sigma} : \nabla \mathbf{v} \, d\Omega = 0.
$$


#### Important notes
It should be noted that the very crucial step here is applying "DIVERGENCE THEOREM" to remove Gradient of Stress Tensor, there by reducing the order of the partial differential equation. Readers interested in a deeper understanding are encourged to read Chapter 6 of the book: "Fish, Jacob, and Ted Belytschko. A first course in finite elements. Vol. 1. New York: Wiley, 2007.". Here is a screenshot of the "DIVERGENCE THEOREM" from Chapter 6 of that book.

<img src="imagines/divergence_theorem.png" />


### Step 3: Relation between Stress and Strain
The stress $\boldsymbol{\sigma}$ depends on the strain $\boldsymbol{\varepsilon}(\mathbf{u})$, where the strain tensor is:

$$
\boldsymbol{\varepsilon}(\mathbf{u}) = \frac{1}{2} \left( \nabla \mathbf{u} + (\nabla \mathbf{u})^T \right).
$$

Similarly, for the test function:

$$
\boldsymbol{\varepsilon}(\mathbf{v}) = \frac{1}{2} \left( \nabla \mathbf{v} + (\nabla \mathbf{v})^T \right).
$$

$$
\nabla \mathbf{v} = \boldsymbol{\varepsilon}(\mathbf{v}) + \frac{1}{2} \left( \nabla \mathbf{v} - (\nabla \mathbf{v})^T \right).
$$

$$\nabla \mathbf{v} = \boldsymbol{\varepsilon}(\mathbf{v}) + \boldsymbol{\omega}(\mathbf{v})$$

where $\boldsymbol{\omega}(\mathbf{v}) = \frac{1}{2} \left( \nabla \mathbf{v} - (\nabla \mathbf{v})^T \right)$, then we have:

$$
\boldsymbol{\sigma} : \nabla \mathbf{v} = \boldsymbol{\sigma} : \boldsymbol{\varepsilon}(\mathbf{v}) + \boldsymbol{\sigma} : \boldsymbol{\omega}(\mathbf{v}),
$$
because $\boldsymbol{\omega}(\mathbf{v}) $ is skew-symmetric and $\boldsymbol{\sigma}$ is symmetric, then $\boldsymbol{\sigma} : \boldsymbol{\omega}(\mathbf{v}) = 0$ (you can verify by yoursefl). Thus:

$$
\int_{\Omega} \boldsymbol{\sigma} : \nabla \mathbf{v} \, d\Omega = \int_{\Omega} \boldsymbol{\sigma} : \boldsymbol{\varepsilon}(\mathbf{v}) \, d\Omega.
$$


#### Julia code to verify that: $\mathbf{A} : \mathbf{B} = \text{tr}\left( \mathbf{A}^T \mathbf{B} \right)=0$. Where $\mathbf{A}$ is symmetric and $\mathbf{B}$ is  skew-symmetric.

In [2]:
using LinearAlgebra
# Define the parameters
a1 = 1;b1 = 2;c1 = 3;d1 = 4;e1 = 5;f1 = 6;
# Build the symmetric matrix
A = [a1 d1 e1;
      d1 b1 f1;
      e1 f1 c1]
# Build the skew-symmetric matrix
B = [0 -a1 -b1;
      a1 0 -c1;
      b1 c1 0]
# Compute sum of dot product of B' and A
inner_product = tr(transpose(A) * B)
println(inner_product)

0



### Step 4: Formulate the Weak Form
Now, we have the weak form as follows:
$$
\int_{\Omega} \boldsymbol{\sigma}(\boldsymbol{\varepsilon}(\mathbf{u})) : \boldsymbol{\varepsilon}(\mathbf{v}) \, d\Omega = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma.
$$


Then, before using Gridap.jl, we define the bilinear form. Find $\mathbf{u}$ such that for all test functions $\mathbf{v}$:

$$
a(\mathbf{u}, \mathbf{v}) = l(\mathbf{v}),
$$

where:

$$
a(\mathbf{u}, \mathbf{v}) = \int_{\Omega} \big[ \boldsymbol{\sigma}(\boldsymbol{\varepsilon}(\mathbf{u})) : \boldsymbol{\varepsilon}(\mathbf{v}) \big] \, d\Omega,
\quad
l(\mathbf{v}) = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma.
$$

## Inputs

In [3]:
L = 20 # Length
W = 10 # Width and Height
domain = (0, L, 0, W, 0, W) # sizes along X, Y, Z
partition = (20, 10, 10) # mesh sizes

(20, 10, 10)

In [4]:
@btime CartesianDiscreteModel(domain, partition);

  9.310 ms (15492 allocations: 2.49 MiB)


### Add labels to left and right faces for boundary conditions

In [5]:
model = CartesianDiscreteModel(domain, partition);
# Tag the left face (x = 0); right face (x = L)
labels = get_face_labeling(model)
add_tag_from_tags!(labels,"left", [1,3,5,7,13,15,17,19,25]); # left face: corners (1,3,5,7); edges (13,15,17,19); others (25) 
add_tag_from_tags!(labels, "right", [2,4,6,8,14,16,18,20,26]); # Right face: corners (2,4,6,8); edges (14,16,18,20); others (26) 
# writevtk(Ω,"Cantilever_Beam_3D")

30-element Vector{String}:
 "tag_01"
 "tag_02"
 "tag_03"
 "tag_04"
 "tag_05"
 "tag_06"
 "tag_07"
 "tag_08"
 "tag_09"
 "tag_10"
 ⋮
 "tag_22"
 "tag_23"
 "tag_24"
 "tag_25"
 "tag_26"
 "interior"
 "boundary"
 "left"
 "right"

### Construct weak form in Gridap.jl

In [7]:
# Integrating on the domain Ω
degree = 2

 Ω = Triangulation(model)
 dΩ = Measure(Ω, degree)
@btime   Triangulation(model)
@btime   Measure(Ω, degree)
# neumann boundaries
Γ = BoundaryTriangulation(model,tags = "right")
dΓ = Measure(Γ,degree)
# Vector-valued FE space
order = 1
reffe = ReferenceFE(lagrangian, VectorValue{3,Float64}, order)

V = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags = "left")
U = TrialFESpace(V)

# Constitutive law for LINEAR ELASTICITY 
const ν = 0.3   # Poisson's ratio
const E = 119e3 # Young's modulus

const λ = (E*ν)/((1+ν)*(1-2*ν)) # C12
const μ = E/(2*(1+ν)) # Shear modulus
ε₀(u) = 0.5 * ( ∇(u) + transpose(∇(u))) # Strain; In Gridap this can be automatically defined in Gridap.ε
σ(ε₀) = λ * tr(ε₀) * one(ε₀) + 2μ * ε₀; # Stress

  183.746 ns (1 allocation: 336 bytes)
  14.500 μs (143 allocations: 10.25 KiB)


## Run finite element analysis

In [8]:
# The weak form
F(x) = VectorValue(0.0, -1, 0.0) # A unit load along Y axis applies to each node on the right face
a(u,v) = ∫((σ∘ε₀(u)) ⊙ ε₀(v)) *dΩ # Left-hand size; (∘) Composite functions
l(v) = ∫(F ⋅ v) * dΓ # Right-hand size
# Solution of the FE problem
op = AffineFEOperator(a,l,U,V)
uh = solve(op)
# writevtk(Ω,"Cantilever_Beam_3D_uh",cellfields=["uh"=>uh,"epsi"=>ε₀(uh),"sigma"=>σ∘ε₀(uh)])
@btime get_matrix(op)

  46.309 ns (1 allocation: 48 bytes)


7260×7260 SparseMatrixCSC{Float64, Int64} with 501642 stored entries:
⎡⠻⣦⡀⠘⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⣀⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⠷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢧⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢶⣄⠈⠻⣦⡀⠙⢷⣄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠙⢷⣄⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⣄⠈⠻⣦⡀⠉⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢷⡄⠈⠻⣦⎦

### Compute the compliance

In [ ]:
# Method 1
C1 = sum(∫((σ ∘ ε₀(uh)) ⊙ ε₀(uh)) * dΩ) # Compliance
println("Compliance 1: ", C1)
# Method 2
C2 = sum(∫(F ⋅ uh) * dΓ) # Compliance
println("Compliance 2: ", C2)
# Method 3
U = get_free_dof_values(uh) # displacement vector
K = get_matrix(op) # stiffness matrix
C3 = dot(U, K * U)
println("Compliance 3: ", C3)
# Method 4 using for loop to compute sum(ue^T*ke*ue)


using Gridap.FESpaces
cell_dofs = get_cell_dof_values(uh)
using Gridap.Arrays
u = get_trial_fe_basis(V)
v = get_fe_basis(V)
K_e = get_array(∫((σ ∘ ε₀(u)) ⊙ ε₀(v)) * dΩ)

numele = num_cells(Ω) # Number of elements

C4 = 0.0 # Comliance
for e=1:numele
    ue = cell_dofs[e]    # element displacement
    ke = K_e[e] # element stiffness matrix
    C4 = C4 + transpose(ue)*ke*ue
end
println("Compliance 4: ", C4)

Compliance 1: 0.30855010386860376
Compliance 2: 0.30855010386853077
Compliance 3: 0.30855010386865317
Compliance 4: 0.30855010386887693


In [15]:
cell_dofs

2000-element LazyArray{FillArrays.Fill{Broadcasting{PosNegReindex{Vector{Float64}, Vector{Float64}}}, 1, Tuple{Base.OneTo{Int64}}}, Vector{Float64}, 1, Tuple{Table{Int32, Vector{Int32}, Vector{Int32}}}}:
 [0.0, -0.00011387182042781587, 0.0, -7.805379263516091e-5, 0.0, -9.628275169810548e-5, 0.0, -6.084372086951238e-5, 0.0, -5.6924146593548694e-5  …  0.0, -3.790205816828376e-5, 0.0, -7.1428903078533e-5, 0.0, -5.654961508551006e-5, 0.0, -4.222616755845123e-5, 0.0, -3.370055061252635e-5]
 [-0.00011387182042781587, -0.00020121582991187485, -7.805379263516091e-5, -0.00014874972877963687, -9.628275169810548e-5, -0.00018959708134364775, -6.084372086951238e-5, -0.00013710794837557135, -5.6924146593548694e-5, -9.904499791513338e-5  …  -3.790205816828376e-5, -9.195853879873363e-5, -7.1428903078533e-5, -9.349515657275712e-5, -5.654961508551006e-5, -7.561241167677022e-5, -4.222616755845123e-5, -6.929511016972387e-5, -3.370055061252635e-5, -5.602070134901073e-5]
 [-0.00020121582991187485, -0.000280

In [14]:
K_e[1]

24×24 Matrix{Float64}:
  27970.1    -12713.7      6356.84   …   -9535.26     -953.526   -4767.63
 -12713.7     27970.1    -10170.9         1907.05     4767.63      953.526
   6356.84   -10170.9     27970.1        -4767.63    -1907.05    -9535.26
 -10170.9      6356.84   -12713.7          953.526    9535.26     1907.05
   6356.84   -10170.9      -635.684      -1907.05    -4767.63     -953.526
 -10170.9      6356.84    -6992.52   …    9535.26      953.526    4767.63
   -635.684   -6992.52     6356.84        -953.526   -9535.26    -1907.05
  -6992.52     -635.684  -10170.9         4767.63     1907.05     9535.26
   9535.26    -1907.05     1907.05        -953.526   -9535.26    -4767.63
   1907.05    -9535.26     9535.26       -1907.05    -4767.63    -9535.26
      ⋮                              ⋱                          
  -4767.63      953.526    -953.526  …    1907.05     4767.63     9535.26
   9535.26    -1907.05     4767.63      -10170.9    -10170.9     -6992.52
   1907.05    -9535.26